# EventSeg-demo

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import LeaveOneOut, KFold, train_test_split
from brainiak.eventseg.event import EventSegment
from hmm_fmri import SimSimpData, Dataset
from utils import download_data, load_data
from pathlib import Path

### Download data

> This will take > 15 seconds

In [ ]:
download_data(Path("data"))

### Load data

In [ ]:
D = load_data(Path("data"), dataset="sherlock")

### Look at some BOLD data

In [ ]:
plt.figure(figsize=(10, 2))
plt.imshow(D["BOLD"][:, :, 1], aspect='auto')
plt.title('Subject 1 BOLD data')
plt.xlabel("TRs")
plt.ylabel("Regions")

### Formal model fitting

This is the nested cross validation skeleton

In [ ]:
# get some variables
nReg, nTRs, nSubs = np.shape(D['BOLD'])

# nested CV constants
n_splits_inner = 4
subj_id_all = np.array(range(nSubs))

# set up outer loop loo structure
loo_outer = LeaveOneOut()
loo_outer.get_n_splits(subj_id_all)
for subj_id_train_outer, subj_id_test_outer in loo_outer.split(subj_id_all):
    print("Outer:\tTrain:", subj_id_train_outer, "Test:", subj_id_test_outer)
    
    # set up inner loop loo structure
    subj_id_all_inner = subj_id_all[subj_id_train_outer]
    kf = KFold(n_splits=n_splits_inner)
    kf.get_n_splits(subj_id_train_outer)
    
    print('Inner:')
    for subj_id_train_inner, subj_id_test_inner in kf.split(subj_id_all_inner):
        # inplace update the ids w.r.t. to the inner training set
        subj_id_train_inner = subj_id_all_inner[subj_id_train_inner]
        subj_id_test_inner = subj_id_all_inner[subj_id_test_inner]
        print("-Train:", subj_id_train_inner, "Test:", subj_id_test_inner, ', now try different k...')
    print()

###  Generate some fake data with `hmm-fmri`

In [ ]:
# create a single data
simpdat = SimSimpData(n_events=15, size=(80, 100), noise=0.3, skew=True, skewf=20).data()

# plot
simpdat.plot()

### Make an entire dataset with `hmm-fmri`

In [ ]:
# create dataset
D = Dataset(base=simpdat, n=40).make_dataset() # dataset with 40 individuals

# plot
dset = np.delete(D.dataset, 0, axis=0)
plt.imshow(dset[0].T, aspect='auto') # first participant data

### Inner loop: Tune `k`

> This is computationally intensive and is scaled by `k` and the training data.

> It is recommended to connect to a remote server or HPC.

In [ ]:
# parse subjects
nSubs = len(D.dataset) - 1
subj_id_test = 0
subj_id_val = 1 
subj_id_train = [
    subj_id for subj_id in range(nSubs) 
    if subj_id not in [subj_id_test, subj_id_val]
]

BOLD_train, BOLD_val_test = train_test_split(dset, test_size=0.4, random_state=42)
BOLD_val, BOLD_test = train_test_split(BOLD_val_test, test_size=0.5, random_state=42) 


# get some k's (adjust as needed)
k_vals = np.arange(1, 15, 1)

# track log-likelihoods
log_likely = []

# fit
for k, j in zip(k_vals, BOLD_train):
    # Fit HMM on training data
    HMM = EventSegment(n_events=k)
    HMM.fit(j)

    # collect LL
    ll = HMM.ll_
    log_likely.append(ll)

# Find the best k based on validation set
# best_k_index = np.argmax(log_likely)
# best_k = k_vals[best_k_index]


print('Whole dataset:\t', np.shape(dset))
print('Training set:\t', np.shape(BOLD_train))
print('Tune set:\t', np.shape(BOLD_val))
print('Test set:\t', np.shape(BOLD_test))

print(subj_id_train)
print(subj_id_val)
print(subj_id_test)

### Plot log-likelyhood

In [ ]:
[plt.plot(i) for i in log_likely];
[plt.legend(i) for i in log_likely]